In [1]:
import requests
import json
import datetime
from datetime import datetime
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import ta
#from ETH_parser import get_floor_price, get_sales_collection, get_sales_token, get_attributes, get_attributes_collection, get_rarity, get_block_timestamp

In [27]:
API_KEY = "vYHU2nVXBMrIHlByeFOQWubnRQcl_v_3"
BAYC = "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d"
AZUKI = "0xed5af388653567af2f388e6224dc7c4b3241c544"

In [3]:
def get_floor_price(API: str, address: str) -> dict:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/getFloorPrice?contractAddress={address}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    prices = {"opensea": data['openSea']['floorPrice'],
              "looksrare": data['looksRare']['floorPrice']}

    return prices

In [4]:
# OLD
# def get_sales_collection(API: str, address: str, limit: int) -> json:
#     url = f"https://eth-mainnet.g.alchemy.com/nft/v3/{API}/getNFTSales?fromBlock=0&toBlock=latest&order=desc&marketplace=seaport&contractAddress={address}&limit={limit}"
#     headers = {"accept": "application/json"}
#     response = requests.get(url, headers=headers)
#     data = json.loads(response.text)

    
#     activity = [{"tokenId": sale["tokenId"],
#                  "marketplace": sale["marketplace"],
#                  "block": sale["blockNumber"],
#                  "token": sale["sellerFee"]["symbol"],
#                  "price": 0 if sale["sellerFee"]["amount"] is None else int(sale["sellerFee"]["amount"]) / 10 ** 18} for sale in data["nftSales"]]

#     return activity

In [59]:
def get_sales_collection(API: str, address: str, to_block, limit: int):
    url = f"https://eth-mainnet.g.alchemy.com/nft/v3/{API}/getNFTSales?fromBlock=0&toBlock={to_block}&order=desc&contractAddress={address}&limit={limit}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    activity = [{"tokenId": sale["tokenId"],
                 "marketplace": sale["marketplace"],
                 "block": sale["blockNumber"],
                 "token": sale["sellerFee"]["symbol"],
                 "price": 0 if sale["sellerFee"]["amount"] is None else int(sale["sellerFee"]["amount"]) / 10 ** 18} for sale in data["nftSales"]]

    return pd.DataFrame(activity)

In [6]:
def get_sales_token(API: str, address: str, limit: int, tokenId: int) -> json:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/getNFTSales?fromBlock=0&toBlock=latest&order=desc&contractAddress={address}&limit={limit}&tokenId={tokenId}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    activity = [{"tokenId": sale["tokenId"],
                 "marketplace": sale["marketplace"],
                 "sellerFee": int(sale["sellerFee"]["amount"]) / 10 ** 18} for sale in data["nftSales"]]
    
    return activity

In [7]:
def get_attributes(API: str, address: str, tokenId: int) -> json:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/computeRarity?contractAddress={address}&tokenId={tokenId}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    return data

In [8]:
def get_attributes_collection(API: str, address: str) -> json:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/summarizeNFTAttributes?contractAddress={address}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    return data['summary']

In [9]:
def get_rarity(API: str, address: str, tokenId: int) -> json:
    data = get_attributes(API=API, address=address, tokenId=tokenId)
    
    result = 1
    for rarity in data:
        result *= rarity["prevalence"]
        
    return result

In [10]:
def get_block_timestamp(block_number):
    url = f"https://eth-mainnet.alchemyapi.io/v2/{API_KEY}"
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [hex(block_number), False],
        "id": 1
    })
    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code == 200:
        timestamp = response.json()['result']['timestamp']
        date_time = datetime.utcfromtimestamp(int(timestamp, 16))
        return date_time
    else:
        return "error"

In [39]:
bayc_data_1 = get_sales_collection(API=API_KEY, address=BAYC, to_block='latest', limit=1000)
next_block = bayc_data_1.block.min()
bayc_data_2 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_2.block.min()
bayc_data_3 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_3.block.min()
bayc_data_4 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_4.block.min()
bayc_data_5 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)

bayc_data = pd.concat([bayc_data_1, bayc_data_2, bayc_data_3, bayc_data_4, bayc_data_5], ignore_index=True)
bayc_data.to_csv('bayc_raw.csv')

,tokenId,marketplace,block,token,price
0,4284,blur,17429314,,46.17
1,428,blur,17429300,,46.17
2,1032,blur,17429285,,46.26
3,3391,blur,17429281,,46.26
4,2578,blur,17429277,,46.33
...,...,...,...,...,...
4995,7525,seaport,16589544,ETH,74.45
4996,3611,blur,16589319,ETH,74.62
4997,3611,x2y2,16589184,ETH,84.39
4998,3634,looksrare,16588074,WETH,66.05


In [45]:
bayc_blocks = bayc_data.copy().drop_duplicates(subset='block')
bayc_blocks['date'] = bayc_blocks['block'].apply(get_block_timestamp)
bayc_blocks.to_csv('bayc_5000.csv')

,tokenId,marketplace,block,token,price
0,4284,blur,17429314,,46.17
1,428,blur,17429300,,46.17
2,1032,blur,17429285,,46.26
3,3391,blur,17429281,,46.26
4,2578,blur,17429277,,46.33
...,...,...,...,...,...
4995,7525,seaport,16589544,ETH,74.45
4996,3611,blur,16589319,ETH,74.62
4997,3611,x2y2,16589184,ETH,84.39
4998,3634,looksrare,16588074,WETH,66.05


# Save Data

In [110]:
bayc_data_1 = get_sales_collection(API=API_KEY, address=BAYC, to_block='latest', limit=1000)
next_block = bayc_data_1.block.min()
bayc_data_2 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_2.block.min()
bayc_data_3 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_3.block.min()
bayc_data_4 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_4.block.min()
bayc_data_5 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_5.block.min()
bayc_data_6 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)
next_block = bayc_data_6.block.min()
bayc_data_7 = get_sales_collection(API=API_KEY, address=BAYC, to_block=next_block, limit=1000)


bayc_data = pd.concat([bayc_data_1, bayc_data_2, bayc_data_3, bayc_data_4, bayc_data_5, bayc_data_6, bayc_data_7], ignore_index=True)
bayc_data.to_csv('bayc_raw.csv')

In [ ]:
bayc_blocks = bayc_data.copy().drop_duplicates(subset='block')
bayc_blocks['date'] = bayc_blocks['block'].apply(get_block_timestamp)
bayc_blocks.to_csv('bayc_w_time.csv')

# Adding tech indicators

In [72]:
# bayc_data = pd.read_csv('bayc_5000.csv').drop('Unnamed: 0', axis=1)
# bayc_raw = pd.read_csv('bayc_raw.csv')

In [92]:
df_main = get_sales_collection(API=API_KEY, address=BAYC, to_block='latest', limit=100)
df_main['date'] = df_main['block'].apply(get_block_timestamp)

In [93]:
dfa = df_main.copy()
dfa

,tokenId,marketplace,block,token,price,date
0,7762,seaport,17436866,ETH,1.49,2023-06-08 16:50:47
1,7833,seaport,17436816,WETH,44.31,2023-06-08 16:40:35
2,7833,seaport,17436816,WETH,45.45,2023-06-08 16:40:35
3,4295,blur,17436568,,45.84,2023-06-08 15:49:47
4,3599,blur,17436306,,45.84,2023-06-08 14:54:47
...,...,...,...,...,...,...
95,5282,blur,17428974,,46.40,2023-06-07 14:05:11
96,428,blur,17428970,,46.40,2023-06-07 14:04:23
97,5101,blur,17428970,,46.40,2023-06-07 14:04:23
98,759,blur,17428968,,46.43,2023-06-07 14:03:59


In [94]:
def calculate_indicators(df):    
    df = df.sort_values('date')  
    
    # SMA
    df['SMA_10'] = ta.trend.sma_indicator(df['price'], window=10)
    df['SMA_20'] = ta.trend.sma_indicator(df['price'], window=20)

    # EMA
    df['EMA_10'] = ta.trend.ema_indicator(df['price'], window=10)
    df['EMA_20'] = ta.trend.ema_indicator(df['price'], window=20)

    # MACD
    df['MACD'] = ta.trend.MACD(df['price']).macd_diff()

    # RSI
    df['RSI'] = ta.momentum.RSIIndicator(df['price']).rsi()

    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df['price'])
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()

    return df

In [95]:
dfa = calculate_indicators(dfa)

In [96]:
dfa

,tokenId,marketplace,block,token,price,date,SMA_10,SMA_20,EMA_10,EMA_20,MACD,RSI,Bollinger_High,Bollinger_Low
99,9816,blur,17428966,,46.43,2023-06-07 14:03:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,759,blur,17428968,,46.43,2023-06-07 14:03:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,428,blur,17428970,,46.40,2023-06-07 14:04:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,5101,blur,17428970,,46.40,2023-06-07 14:04:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,5282,blur,17428974,,46.40,2023-06-07 14:05:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,3599,blur,17436306,,45.84,2023-06-08 14:54:47,45.76,45.66,45.77,45.58,-0.07,55.18,46.08,45.25
3,4295,blur,17436568,,45.84,2023-06-08 15:49:47,45.78,45.69,45.78,45.60,-0.07,55.18,46.09,45.29
2,7833,seaport,17436816,WETH,45.45,2023-06-08 16:40:35,45.77,45.71,45.72,45.59,-0.09,50.27,45.98,45.44
1,7833,seaport,17436816,WETH,44.31,2023-06-08 16:40:35,45.63,45.65,45.47,45.47,-0.18,39.27,46.31,44.99


In [107]:
def calcilate_intersections(df):
    # sma crossover
    df['signal_sma'] = 'no signal'
    df['sma_diff'] = df['SMA_10'] - df['SMA_20']
    df.loc[(df['sma_diff'] > 0) & (df['sma_diff'].shift(1) < 0), 'signal_sma'] = 'buy'
    df.loc[(df['sma_diff'] < 0) & (df['sma_diff'].shift(1) > 0), 'signal_sma'] = 'sell'

    # ema crossover
    df['signal_ema'] = 'no signal'
    df['ema_diff'] = df['EMA_10'] - df['EMA_20']
    df.loc[(df['ema_diff'] > 0) & (df['ema_diff'].shift(1) < 0), 'signal_ema'] = 'buy'
    df.loc[(df['ema_diff'] < 0) & (df['ema_diff'].shift(1) > 0), 'signal_ema'] = 'sell'

    # macd and rsi
    df['mr_signal'] = 'no signal'
    df.loc[(df['MACD'] > 0) & (df['RSI'] < 30), 'mr_signal'] = 'buy'
    df.loc[(df['MACD'] < 0) & (df['RSI'] > 70), 'mr_signal'] = 'sell'

    # mbb
    df['mbb_signal'] = 'no signal'
    df.loc[(df['MACD'] > 0) & (df['price'] <= df['Bollinger_Low']), 'mbb_signal'] = 'buy'
    df.loc[(df['MACD'] < 0) & (df['price'] >= df['Bollinger_High']), 'mbb_signal'] = 'sell'

    # rbb
    df['rbb_signal'] = 'no signal'
    df.loc[(df['RSI'] < 30) & (df['price'] <= df['Bollinger_Low']), 'rbb_signal'] = 'buy'
    df.loc[(df['RSI'] > 70) & (df['price'] >= df['Bollinger_High']), 'rbb_signal'] = 'sell'

    return dfa

In [108]:
dfa = calcilate_intersections(dfa)
dfa

,tokenId,marketplace,block,token,price,date,SMA_10,SMA_20,EMA_10,EMA_20,...,RSI,Bollinger_High,Bollinger_Low,signal_sma,sma_diff,signal_ema,ema_diff,mr_signal,mbb_signal,rbb_signal
99,9816,blur,17428966,,46.43,2023-06-07 14:03:35,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,no signal,NaN,no signal,NaN,no signal,no signal,no signal
98,759,blur,17428968,,46.43,2023-06-07 14:03:59,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,no signal,NaN,no signal,NaN,no signal,no signal,no signal
96,428,blur,17428970,,46.40,2023-06-07 14:04:23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,no signal,NaN,no signal,NaN,no signal,no signal,no signal
97,5101,blur,17428970,,46.40,2023-06-07 14:04:23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,no signal,NaN,no signal,NaN,no signal,no signal,no signal
95,5282,blur,17428974,,46.40,2023-06-07 14:05:11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,no signal,NaN,no signal,NaN,no signal,no signal,no signal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,3599,blur,17436306,,45.84,2023-06-08 14:54:47,45.76,45.66,45.77,45.58,...,55.18,46.08,45.25,no signal,0.10,no signal,0.20,no signal,no signal,no signal
3,4295,blur,17436568,,45.84,2023-06-08 15:49:47,45.78,45.69,45.78,45.60,...,55.18,46.09,45.29,no signal,0.09,no signal,0.18,no signal,no signal,no signal
2,7833,seaport,17436816,WETH,45.45,2023-06-08 16:40:35,45.77,45.71,45.72,45.59,...,50.27,45.98,45.44,no signal,0.06,no signal,0.14,no signal,no signal,no signal
1,7833,seaport,17436816,WETH,44.31,2023-06-08 16:40:35,45.63,45.65,45.47,45.47,...,39.27,46.31,44.99,sell,-0.03,no signal,0.00,no signal,no signal,no signal
